In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [2]:
from case_code import CodeDownloader, get_metadata, CodeLoader, transform, Case

# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()
print(f"{'Total files downloaded:':<24} {downloaded_count}")

metadata = get_metadata()
print(f"{'Total cases in metadata:':<24} {len(metadata)}")

Total files downloaded:  125
Total cases in metadata: 110


In [3]:
from utils.model_selector import get_chat_model
import pandas as pd

# Get the chat model
model = get_chat_model("openai")

# Transform the downloaded code to case
loader = CodeLoader()
skipped = await transform(model=model, loader=loader, total=downloaded_count)

data = [(key.value, len(value)) for key, value in skipped.items()]
df = pd.DataFrame(data, columns=["Error", "count"])
print(df)

Processing: 100%|██████████| 125/125 [08:14<00:00,  3.95s/it]

            Error  count
0  Case Not Found     57
1     Parse Error      1


In [4]:
from case import get_retriever

In [5]:
get_retriever().invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 2, 'source': '/Users/clement/Developer/ai/tx-generator/data/case_gpt-4o.jsonl', 'steps': "[{'description': 'ETH to Lido', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '{user_input}', 'function_name': 'submit', 'input_args': ['ReferalAccount']}, {'description': 'Approve stETH to Eigenpie', 'to': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'value': '0', 'function_name': 'approve', 'input_args': ['0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', '{user_input}']}, {'description': 'stETH to Eigenpie', 'to': '0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', 'value': '0', 'function_name': 'depositAsset', 'input_args': ['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', '{user_input}', '{user_input}', 'ReferalAccount']}]", 'total_steps': 3}, page_content='Staking ETH with Lido and then depositing stETH to Eigenpie.'),
 Document(metadata={'case_id': 'lido_multiplier', 'seq_num': 53, 'source': '/Users/clement/Developer/ai/tx-genera

In [7]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [8]:
question = "Stake ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
result

Case(case_id='eigenpie_steth', description='Stake ETH with Lido and deposit to Eigenpie', total_steps=3, steps=[Tx(description='ETH to Lido', to='0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', value='{user_input}', function_name='submit', input_args=['ReferalAccount']), Tx(description='Approve stETH to Eigenpie', to='0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', value='0', function_name='approve', input_args=['0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', '{user_input}']), Tx(description='stETH to Eigenpie', to='0x24db6717dB1C75B9Db6eA47164D8730B63875dB7', value='0', function_name='depositAsset', input_args=['0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', '{user_input}', '{user_input}', 'ReferalAccount'])])

In [12]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
result

Case(case_id='yield_farming_multiple_protocols', description='Yield Farming with Multiple Protocols', total_steps=4, steps=[Tx(description='Deposit ETH into AAVE to earn interest', to='0x893411580e590D62dDBca8a703d61Cc4A8c7b2b9', value='{user_input}', function_name='depositETH', input_args=['0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', '{user_input}', '0']), Tx(description='Use aToken (aETH) as collateral to borrow DAI', to='0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', value='0', function_name='borrowDAI', input_args=['0xDAI_address', '{user_input}']), Tx(description='Deposit the borrowed DAI into Yearn Finance', to='0xYearnFinance_address', value='{user_input}', function_name='deposit', input_args=['DAI', '{user_input}']), Tx(description='Use Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees', to='0xUniswap_address', value='{user_input}', function_name='addLiquidity', input_args=['Yearn_vault_tokens', '{user_input}'])])